In [1]:
import polars as pl
import numpy as np
import requests

In [2]:
tkv_df = pl.read_csv('all_tkv.csv', schema_overrides={'project_id': pl.String})

In [3]:
tkv_df.columns

['transaction_id',
 'created',
 'updated',
 'instrument_id',
 'project_id',
 'display_name',
 'value']

In [4]:
tkv_df['display_name'].unique()

display_name
str
"""Dataset Type (DMS)"""
"""Number of Scans (DMS)"""
"""Tag1"""
"""Tag"""
"""Experiment Name (DMS)"""
…
"""Session ID (PNCC)"""
"""Session ID (Krios)"""
"""Microscope Run Start Date (PNC…"


In [5]:
pivoted_df = tkv_df[['transaction_id','display_name','value']].pivot(index='transaction_id', columns='display_name', values='value', aggregate_function='first')

/var/folders/zj/15rg7d7j4v9b5cj3mlzqz9xc0000gn/T/ipykernel_92974/2412600687.py:1: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  pivoted_df = tkv_df[['transaction_id','display_name','value']].pivot(index='transaction_id', columns='display_name', values='value', aggregate_function='first')


In [6]:
pivoted_df = pivoted_df[['transaction_id','Dataset (DMS)', 'Organism', 'Taxonomy ID (NCBI)', 'Tag']]

In [7]:
pivoted_df = pivoted_df.rename({'Dataset (DMS)': 'dataset_name', 'Organism': 'organism', 'Taxonomy ID (NCBI)': 'taxonomy_id', 'Tag': 'description'})

In [8]:
pivoted_df

transaction_id,dataset_name,organism,taxonomy_id,description
i64,str,str,str,str
778524,"""PSA1000pg_1st_D_500ugserum_fra…","""Homo_sapiens""","""9606""",null
1223630,"""2017_04_13_Kyle_I_Tannic_AGW_K…","""None""","""None""",null
1008331,"""QC_CHECKpreclean_23Jun15_r5""","""Homo_sapiens""","""9606""",null
924881,"""OMICS_WLN002_mock_pool_12_18De…","""Homo_sapiens""","""9606""",null
924880,"""OMICS_WLN002_mock_pool_12_18De…","""Homo_sapiens""","""9606""",null
…,…,…,…,…
1239692,null,null,null,null
270313,null,null,null,null
221748,null,null,null,null


In [9]:
tkv_df = tkv_df[['transaction_id', 'created', 'updated', 'instrument_id', 'project_id']]

In [10]:
tkv_df = tkv_df.group_by('transaction_id', 'created', 'updated', 'instrument_id', 'project_id').agg()

In [11]:
tkv_cleaned_df = pivoted_df.join(tkv_df, on='transaction_id', how='left')

In [12]:
tkv_cleaned_df

transaction_id,dataset_name,organism,taxonomy_id,description,created,updated,instrument_id,project_id
i64,str,str,str,str,str,str,i64,str
778524,"""PSA1000pg_1st_D_500ugserum_fra…","""Homo_sapiens""","""9606""",null,"""13/6/2015 10:10:30""","""4/10/2018 08:12:09""",34215,"""48505"""
1223630,"""2017_04_13_Kyle_I_Tannic_AGW_K…","""None""","""None""",null,"""17/4/2017 16:05:08""","""17/4/2017 16:05:08""",34011,"""49712"""
1008331,"""QC_CHECKpreclean_23Jun15_r5""","""Homo_sapiens""","""9606""",null,"""21/5/2016 20:23:30""","""21/5/2016 20:23:30""",34234,"""48505"""
924881,"""OMICS_WLN002_mock_pool_12_18De…","""Homo_sapiens""","""9606""",null,"""14/1/2016 14:13:37""","""14/1/2016 14:13:37""",34116,"""48199"""
924880,"""OMICS_WLN002_mock_pool_12_18De…","""Homo_sapiens""","""9606""",null,"""14/1/2016 14:13:23""","""14/1/2016 14:13:23""",34116,"""48199"""
…,…,…,…,…,…,…,…,…
1239692,null,null,null,null,"""9/5/2017 16:47:18""","""9/5/2017 16:47:18""",34127,"""48680"""
270313,null,null,null,null,"""17/8/2013 23:18:59""","""17/8/2013 23:18:59""",34134,"""39968"""
221748,null,null,null,null,"""10/3/2013 20:49:46""","""10/3/2013 20:49:46""",34176,"""39968"""


In [13]:
dois_df = pl.read_csv("dois.csv")

In [14]:
dois_df['site_url'][0]

'https://release.my.emsl.pnnl.gov/released_data/1850470'

In [15]:
datasets_df = tkv_cleaned_df.join(dois_df, how="left", left_on="transaction_id", right_on="upload_id")

In [16]:
datasets_df

transaction_id,dataset_name,organism,taxonomy_id,description,created,updated,instrument_id,project_id,doi,site_url
i64,str,str,str,str,str,str,i64,str,str,str
778524,"""PSA1000pg_1st_D_500ugserum_fra…","""Homo_sapiens""","""9606""",null,"""13/6/2015 10:10:30""","""4/10/2018 08:12:09""",34215,"""48505""",null,null
1223630,"""2017_04_13_Kyle_I_Tannic_AGW_K…","""None""","""None""",null,"""17/4/2017 16:05:08""","""17/4/2017 16:05:08""",34011,"""49712""",null,null
1008331,"""QC_CHECKpreclean_23Jun15_r5""","""Homo_sapiens""","""9606""",null,"""21/5/2016 20:23:30""","""21/5/2016 20:23:30""",34234,"""48505""",null,null
924881,"""OMICS_WLN002_mock_pool_12_18De…","""Homo_sapiens""","""9606""",null,"""14/1/2016 14:13:37""","""14/1/2016 14:13:37""",34116,"""48199""",null,null
924880,"""OMICS_WLN002_mock_pool_12_18De…","""Homo_sapiens""","""9606""",null,"""14/1/2016 14:13:23""","""14/1/2016 14:13:23""",34116,"""48199""",null,null
…,…,…,…,…,…,…,…,…,…,…
1239692,null,null,null,null,"""9/5/2017 16:47:18""","""9/5/2017 16:47:18""",34127,"""48680""",null,null
270313,null,null,null,null,"""17/8/2013 23:18:59""","""17/8/2013 23:18:59""",34134,"""39968""",null,null
221748,null,null,null,null,"""10/3/2013 20:49:46""","""10/3/2013 20:49:46""",34176,"""39968""",null,null


In [17]:
datasets_df = datasets_df.with_columns(("https://release.my.emsl.pnnl.gov/released_data/" + datasets_df['transaction_id'].cast(pl.String)).alias("site_url"))

In [18]:
datasets_df = datasets_df.with_columns(pl.lit("EMSL").alias("data_source"), pl.lit(46.348660).alias("latitude"), pl.lit(-119.278173).alias("longitude"))

In [19]:
datasets_df.columns

['transaction_id',
 'dataset_name',
 'organism',
 'taxonomy_id',
 'description',
 'created',
 'updated',
 'instrument_id',
 'project_id',
 'doi',
 'site_url',
 'data_source',
 'latitude',
 'longitude']

In [20]:
datasets_df = datasets_df.rename({'transaction_id': 'id', 'dataset_name': 'name', 'created':'date_created', 'updated': 'date_updated', 'site_url': 'url' })

In [21]:
vals = datasets_df[['organism', 'taxonomy_id', 'instrument_id']].to_dicts()

In [22]:
cleaned_vals = []
for val in vals:
    cleaned_val = {}
    for key in val:
        if val[key] != 'None':
            cleaned_val[key] = val[key]
    cleaned_vals.append(cleaned_val)

In [28]:
datasets_dict = datasets_df[["id", "name", "description","date_created","date_updated","instrument_id","project_id","doi","url","data_source","latitude","longitude"]].to_dicts()

In [30]:
new_vals = []
for val, metadata in zip(datasets_dict, cleaned_vals):
    new_val = val
    new_val['metadata'] = metadata
    new_vals.append(new_val)

In [ ]:
import json
with open('emsl_datasets.json', 'w') as f:
    f.write(json.dumps(new_vals))